<span style="font-size: 2em;">Gerekli kütüphanelerin indirilmesi</span>


In [1]:
#!pip install opencv-python

In [2]:
#!pip install deepface

In [3]:
#!pip install pyttsx3

In [4]:
#!pip install SpeechRecognition

In [5]:
#!pip install pygame

In [6]:
#!pip install tensorflow

In [7]:
#!pip install opencv-python --upgrade

In [8]:
#!pip install deepface --upgrade

In [9]:
#!pip install PyAudio

In [1]:
import torch
print(torch.cuda.is_available())  
print(torch.cuda.device_count()) 
print(torch.cuda.get_device_name(0))  

True
1
NVIDIA GeForce RTX 2060


In [ ]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset

# 1. Model ve tokenizer'ı yükle
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
model = GPT2LMHeadModel.from_pretrained("./trained_model")

In [ ]:
model.eval()

<span style="font-size: 2em;">Chat-Bot fonksiyonları</span>

In [10]:
import speech_recognition as sr
import pyttsx3
import datetime
import pygame
import webbrowser
import random
import requests
import time
isim = 'yok'
# Ufuk veya Gülsüm son olarak ne zaman tespit edildiğini tutan zaman damgası
last_seen_time = time.time()
# Belirli bir süre boyunca kimse tespit edilmezse uygulamanın kapanması için maksimum süre (saniye cinsinden)
max_unseen_duration = 20

günler = {
    "Monday": "Pazartesi",
    "Tuesday": "Salı",
    "Wednesday": "Çarşamba",
    "Thursday": "Perşembe",
    "Friday": "Cuma",
    "Saturday": "Cumartesi",
    "Sunday": "Pazar"
}
stop_bot = False
api_key = "API KEY"

def speak(text):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    engine.setProperty('voice', 'turkish')
    engine.say(text)
    engine.runAndWait()
    print(text)

def play_number_game():
    secret_number = random.randint(1, 100)
    speak("Bir ile yüz arasında bir sayı tuttum. Tahmin et")
    print("Bir ile yüz arasında bir sayı tuttum. Tahmin et")
    while True:
        query = listen()
        if "çıkış" in query:
            speak(f"Oyun bitti. Doğru sayı {secret_number} idi.")
            print(f"Oyun bitti. Doğru sayı {secret_number} idi.")
            break

        if "tahmin" in query:
            guess = int([int(s) for s in query.split() if s.isdigit()][0])
            if guess == secret_number:
                speak("Tebrikler! Doğru tahmin ettiniz.")
                print("Tebrikler! Doğru tahmin ettiniz.")
                break
            elif guess < secret_number:
                speak("Daha yüksek bir sayı söyleyin.")
                print("Daha yüksek bir sayı söyleyin.")
            else:
                speak("Daha düşük bir sayı söyleyin.")
                print("Daha düşük bir sayı söyleyin.")
        else:
            speak("Lütfen bir sayı söyleyin veya çıkmak için 'çıkış' deyin.")
            print("Lütfen bir sayı söyleyin veya çıkmak için 'çıkış' deyin.")
    
def get_weather(api_key):
    city_name = "Kirsehir"
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city_name,
        "appid": api_key,
        "units": "metric",  # Metrik birimlerde hava durumu almak için
        "lang": "tr"        # Türkçe dilinde hava durumu almak için
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather_desc = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        result = f"Kırşehir'de hava durumu: {weather_desc}. Sıcaklık: {temp}°C."
        print(result)
        return result
    else:
        return "Hava durumu bilgisi alınamadı."

def play_music():
    pygame.init()
    pygame.mixer.music.load('music1.mp3')
    pygame.mixer.music.play()


def listen(timeout=5, retries=3):
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        print("Sizi dinliyorum... Lütfen konuşun.")
        for _ in range(retries):
            try:
                # timeout süresi boyunca ses dinleme
                audio = recognizer.listen(source, timeout=timeout)
                print("Ses algılanıyor, lütfen bekleyin...")
                # Google Web Speech API kullanarak sesi metne çevirme
                text = recognizer.recognize_google(audio, language='tr-TR')
                print("Ses algılandı: ", text)
                return text.lower()
            except sr.WaitTimeoutError:
                # Belirtilen timeout süresi içinde ses algılanmadı
                print("Ses algılanamadı, yeniden deniyor...")
                continue  # Döngüyü tekrar başlat
            except sr.UnknownValueError:
                print("Ses anlaşılamadı, lütfen tekrar deneyin.")
                continue
            except sr.RequestError as e:
                print(f"Google API'dan yanıt alınamadı, hata: {e}")
                continue
        return "Anlaşılmadı"

def chatbot_response(query):
    global stop_bot
    print("Sohbet botuna hoş geldiniz! Şu anda sadece İngilizce sohbet geçerlidir. (Çıkmak için 'exit' yazın)")

    while True:
        # Kullanıcıdan bir soru al
        user_input = input("Sen: ")
        
        # Çıkış için 'exit' komutunu kontrol et
        if user_input.lower() == "exit":
            print("Sohbet sona erdi.")
            break

        # Kullanıcı girdisini tokenleştir
        inputs = tokenizer(user_input, return_tensors="pt")
        
        # Model ile yanıt üret
        output = model.generate(
            inputs.input_ids, 
            max_length=100,  # Üretilecek maksimum token sayısı
            num_return_sequences=1,  # Kaç yanıt üretileceği
            no_repeat_ngram_size=2,  # Aynı cümlenin tekrarını önlemek için
            early_stopping=True
        )

        # Tokenleri metne dönüştür
        response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Modelin yanıtını göster
        print(f"Bot: {response}")

# Sohbet başlat
chat_with_model()


pygame 2.6.1 (SDL 2.28.4, Python 3.9.20)
Hello from the pygame community. https://www.pygame.org/contribute.html


<span style="font-size: 2em;">Yüz tanıma işlemleri</span>

In [11]:
import threading
import cv2
from deepface import DeepFace
import time
import pyttsx3
import speech_recognition as sr

stop_threads = threading.Event()
engine = pyttsx3.init()
is_spoken = False
stop_bot = False
bot_responded = False
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

counter = 0

reference_img1 = cv2.imread("ufukface.jpeg")
reference_img2 = cv2.imread("secondface.jpg")

face_match1 = False
face_match2 = False


def check_face1(frame):
    global face_match1
    try:
        if DeepFace.verify(frame, reference_img1.copy())['verified']:
            face_match1 = True
        else:
            face_match1 = False
            
    except ValueError:
        face_match1 = False

def check_face2(frame):
    global face_match2
    try:
        if DeepFace.verify(frame, reference_img2.copy())['verified']:
            face_match2 = True
            
        else:
            face_match2 = False
            
    except ValueError:
        face_match2 = False

def chatbot_thread_func():
    global stop_bot
    while not stop_bot:
        query = listen()
        response = chatbot_response(query)
        speak(response)
        if stop_bot:
            break

chatbot_thread = threading.Thread(target=chatbot_thread_func)

while True:
    bot_responded = False
    if stop_bot:
        stop_bot = False
        chatbot_thread.join()
        
    ret, frame = cap.read()

    if ret:
        if counter % 30 == 0:
            if not stop_threads.is_set():
                try:
                    threading.Thread(target=check_face1, args=(frame.copy(),)).start()
                    threading.Thread(target=check_face2, args=(frame.copy(),)).start()
                except ValueError:
                    pass
        counter += 1

        if face_match1:
            cv2.putText(frame, "Ufuk!", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 1)
            isim = 'Ufuk'
            last_seen_time = time.time()
            #print(isim)
            if not chatbot_thread.is_alive():
                chatbot_thread = threading.Thread(target=chatbot_thread_func)
                chatbot_thread.start()
            
        else:
            stop_threads.clear()
            cv2.putText(frame, "eslesme yok 1", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 1)
            
        if face_match2:
            cv2.putText(frame, "SeconFace!", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 1)
            isim = 'Second'
            last_seen_time = time.time()
            if not chatbot_thread.is_alive():
                chatbot_thread = threading.Thread(target=chatbot_thread_func)
                chatbot_thread.start()
            
        else:
            cv2.putText(frame, "eslesme yok 2", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 1)
        
        cv2.imshow('video', frame)
    if (time.time() - last_seen_time) > max_unseen_duration:
        stop_bot = True
        

    #remaining_time = max_unseen_duration - (time.time() - last_seen_time)
    #print(f"Kapanmaya kalan süre: {int(remaining_time)} saniye")  # Kalan süreyi saniye cinsinden yazdır
    
    key = cv2.waitKey(1)
    if key == ord('q') or stop_bot:
        break

cv2.destroyAllWindows()
cap.release()



Sizi dinliyorum... Lütfen konuşun.
Ses algılanıyor, lütfen bekleyin...
Ses algılandı:  nasılsın
İyiyim, teşekkür ederim. sen nasılsın?
Sizi dinliyorum... Lütfen konuşun.
Ses algılanamadı, yeniden deniyor...
Ses algılanamadı, yeniden deniyor...
Ses algılanıyor, lütfen bekleyin...
Ses algılandı:  iyi misin


In [12]:
#!pip install pyowm